_Комментарий_ : данный ноутбук не запущен, поскольку многие вычисления выполнялись в разных ноутбуках и не единовременно. Таким образом, данный ноутбку является обобщающим и поясняющим весь ход работы. Полностью обработанный датасет можно найти на гитхабе в файле READ.ME.  
Также в данном ноутбуке полностью опущена работа с датасетом (соединение отедльных файлов, добавление колонки с обработанными данными, склеивание датасетов и тд.)

# Очиста данных и обработка текста

Удаляем новости, датируемые 1914 годом.

In [ ]:
df_full = pd.read_csv("lenta-ru-news.csv")
df_full.head()

In [ ]:
# переводим дату в формат даты
df_full['date'] = pd.to_datetime(df_full['date'], format = '%Y-%m-%d')

# удаляет 2014 год
df = df_full[df_full['date'].dt.year != 1914]
df.reset_index(inplace=True, drop=True)
df

Импорт нужных библиотек и моделей для обработки текста новостей

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import nltk
from tqdm import tqdm

In [ ]:
from natasha import Doc
from natasha import Segmenter
segmenter = Segmenter()

import string
print(string.punctuation)
spec_chars = string.punctuation + '\n«»\t—…–№' 
spec_chars

from nltk.corpus import stopwords
unique_stops = set(stopwords.words('russian'))

import pymorphy2
morph = pymorphy2.MorphAnalyzer()

Поскольку наши данные представлены в виде связного текста, нам необходимо преобразовать их в понятный для машины язык - набор векторов. Перед этим необходимо провести обработку всех текстов и преобразовать их в массивы слов в начальной форме.

**Общий подход**  
>Подготовка данных для последующей кодировки была произведена двумя путями. Основное отличие подходов заключалось в разных принципах работы лемматизаторов (библиотеки pymystem3 и pymorphy2).
>
>Выбранные библиотеки были разработаны для обработки текстов русского языка. На данном этапе отпала всемиизвестная библиотека nltk. По визуальному анализу токенизированных текстов было видно, что она работает на порядок хуже.
>
>Отличие между этими библиотеками состоит в том, что pymystem3 токенизирует и лемматизирует слова с учетом контекста слова, что является большим преимуществом для русского языка. А библитека pymorphy2 лемматизиует слова уже после токенизации. Ее приятная черта в том, что можно настроить лемматизацию вручную.


____
Данный ноутбук содержит обработку текстов с помощью библиотеки - **pymorphy2**. Как уже было сказано ранее, в отличие от Mystem данный лемматизатор определяет часть речи и приводит к начальной форме без учета контекста слова. Это имеет свои недостатки (например фраза `сделан из стали` будет выглядеть как `"сделать", "из", "стать"`, а нет `"сталь"`, как следует из контекста).  
___
Также эта библиотека работает уже с токенизированными словами. Таком образом, предварительно требуется токенизация текста.

Здесь был выбор из двух библиотек: классическая nltk и русская natasha. Проводилось тестирование работы обоих библиотек на небольшой части датасета и применением после визуального анализа результатов. Практически сразу было видно явное преимущество по качеству библиотеки **natasha**, разработанную в России специально для обработки текстов русского языка. 
>Примером может послужить следующая фраза: `«битвы…Величие»`. Фраза взята из одной новости. К сожалению, подобные опечатки довольно часто встречались в датасете. Библиотека nltk не распознавала из и после токенизации они так и оставались «слипшимися». Однако Natasha спокойно с этим справлялась и выдавала `«битвы» и «Величие»`.


Функция, которая обрабатывает текст. На вход принимает текст одной новости.

Обработка текста была выстроена по следующей логике:
- импорты библиотек, потому что при обработке было применено распараллеливание и функции импортированные глобально не видны
- если новость была пустой (nan), обработка не производилась и возвращалось значение "\$\_\$nan\$\_\$"
- токенизация
- удаление лишних символов (по типу !"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~/n«»\t—…–№)
- удаление стоп-слов (словарь стоп-слов был взят из библиотеки nltk)
- лемматизация с помощью библиотеки pymorphy

Отдельное приведение в нижний регистр слов отсутствовало, поскольку Natasha при токенизации автоматически переводит слова в нижний регистр.

In [ ]:
def word_processing(text):
    from natasha import Doc
    from natasha import Segmenter
    segmenter = Segmenter()

    import string
    #print(string.punctuation)
    spec_chars = string.punctuation + '\n«»\t—…–№' 
    spec_chars


    from nltk.corpus import stopwords
    unique_stops = set(stopwords.words('russian'))

    import pymorphy2
    morph = pymorphy2.MorphAnalyzer()
    
    if str(text) == 'nan':
        return '$_$nan$_$'
    
    else:
        # переводим текст в формат наташи
        doc = Doc(text)

        # разбиение на токены
        doc.segment(segmenter)
        #doc.tokens

        # Очистка списка от лишних знаков (подумать над двойными кавычками: '', "", ``) --upd убрала тупым образом
        doc_natasha = []
        for i in range(len(doc.tokens)):
            if doc.tokens[i].text not in spec_chars:
                doc_natasha.append(doc.tokens[i].text)


        # удаление стоп-слов и перевод в нижний регистр
        no_stops = []
        for token in doc_natasha:
            token1 = token.lower()
            if token1 not in unique_stops: #and token.isalpha():
                no_stops.append(token1)

        # сложная лемматизация
        doc_lem = []
        for i in range(len(no_stops)):
            word1 = morph.parse(no_stops[i])[0]
            if ('INFN' in word1.tag) or ('VERB' in word1.tag) or ('GRND' in word1.tag) or ('PRTF' in word1.tag) or ('PRTS' in word1.tag):
                word = word1.normalized.word
                doc_lem.append(word)
            else:
                try:
                    word = word1.inflect({'nomn'}).word
                    doc_lem.append(word)
                except:
                    word = word1.normal_form
                    doc_lem.append(word)

        return doc_lem

Импорт библиотеки для распараллеливания.

In [ ]:
from multiprocess import Pool

Далее расчеты производились следующим образом:  
Поскольку файл очень объемный на каждой итерации цикла считывалось определенное кол-во строк из датасета, которые затем посылались в вышеописанную функцию (+ применялось распараллеливания для ускорения работы). Затем после обработки этого пула строк они в виде датафрейма сохранялись в csv, чтобы избежать потерь на случай прерывании работы функции.

In [ ]:
df_short = pd.DataFrame(columns=['text'])
df_short.to_csv('file_for_save.csv', index=False)

In [ ]:
%%time


for i in tqdm(range(5)):
    # Считывание текста по кускам
    my_df = pd.read_csv("lenta-ru-news.csv", nrows=100, header=0, skiprows=range(1, 10000*2+100*i*2+1))
    news_text = my_df['text']
    test=[]
   

    with Pool(processes=4) as pool:
        for i in tqdm(pool.imap(word_processing, iter(news_text))):
            test.append(i)
            df_short.at[len(test)-1, 'text'] = i

    df_short.to_csv('file_for_save.csv', index=False, mode='a', header=False)
    df_short.drop(df_short.index[:], inplace=True)

Функция обрабатывала 10тыс текстов за 8.30 минут.

## Дополнения

Изначально функция имела другой вид. Это было связано с тем, что в датасете имелись плохо написанные слова (например "пустыниСражались"), то есть они были слипшиеся. Для исправления этой проблемы было решено использовать **YandexSpeller**.

Однако потом пришлось отказаться от его применения по нескольким причинам. Во-первых, это было очень это было очень времязатратно, потому что спеллер мог исправлять ошибки только на небольшом фрагменте текста и приходилось ему давать по одному слову из списка из токенизированных слов. Во-вторых, не ней стоит ограничение по кол-посылок в день (оно довольно небольшое). 

Функция работала слишком долго (весь датасет обработался примерно бы за 8 суток), поэтому от исправления ошибок пришлось отказаться и пожертвовать "слипшимися" словами.